# Surface Navier-Stokes

In this tutorial we will solve the surface Navier-Stokes equations with the H(div)-conforming HDG method, [<a href="https://onlinelibrary.wiley.com/doi/10.1002/nme.6317">Lederer, Lehrenfeld, Schöberl: Divergence-free Tangential Finite Element Methods for Incompressible Flows on Surfaces, <i>Int J Numer Methods Eng</i> (2020).</a>]. 
See also Tutorial `3.5 Scalar PDE on surfaces (with HDG)`.


Find $(u,p)\in H^1(S,TS)\times L^2(S)$ such that 

\begin{align*}
&\partial_t u - 2\overline{\nu} \boldsymbol{P}\mathrm{div}_S(\varepsilon_S(u)) + (u\cdot\nabla_S)u + \nabla_S p = f,\\
	&\mathrm{div}_S(u)=0.
\end{align*}

Here, $\boldsymbol{P}=\boldsymbol{I}-\nu\otimes\nu$ is the projection onto the tangent space, $\varepsilon_S(u)= 0.5(\boldsymbol{P}\nabla_S u + (\boldsymbol{P}\nabla_Su)^T)$ the symmetric covariant gradient, and $\mathrm{div}_S(u)=\mathrm{tr}(\nabla_S u)$. 

The weak form is given by: Find $(u,p)\in H^1(S,TS)\times L^2(S)$ such that for all $(v,q)\in H^1(S,TS)\times L^2(S)$

\begin{align*}
&\int_S\partial_t u\cdot v + \overline{\nu}\, \varepsilon_S(u):\varepsilon_S(v) + (u\cdot\nabla_S)u\cdot v - \mathrm{div}_S(v)\, p\,ds = \int_Sf\cdot v\,ds\\
&-\int_S\mathrm{div}_S(u)\,q\,ds=0
\end{align*}
or in bilinear form notation with mass $m(\cdot,\cdot)$, stiffness $a(\cdot,\cdot)$, convection $c(\cdot,\cdot,\cdot)$ , and divergence constraint $b(\cdot,\cdot)$

\begin{align*}
&m(\partial_t u,v) + a(u,v) + c(u,u,v) + b(v,p) = f(v),\\
&b(u,q) = 0.
\end{align*}

### H(div)-conforming HDG
The velocity will be approximated by H(div)-conforming elements mapped onto the surface $V^k_h$. Thus, they are co-normal continuous. The tangential-continuity will be enforced in terms of HDG by a tangential facet FESpace $\mathcal{F}^k_h$ living only on the skeleton (edges)

\begin{align*}
&\tilde{V}^k_h = V_h^k\times \mathcal{F}^k_h,\\
&\tilde{u}_h = (u_h,\hat{u}_h)\in \tilde{V}_h.
\end{align*}

<img src="pictures/hdg_normal_cont_trigs.jpg" width="150 align="center"> 



The stiffness bilinear form reads

\begin{align*}
a_h(\tilde{u}_h,\tilde{v}_h)= \sum_{T\in\mathcal{T}_h} &\int_T \overline{\nu}\,\varepsilon_S(u_h):\varepsilon_S(v_h)\,ds + \int_{\partial T} \overline{\nu}(\varepsilon_S(u_h)\mu)\cdot (\hat{v}_h-v_h)^{\tau}\,dl+ \int_{\partial T} \overline{\nu}(\varepsilon_S(v_h)\mu)\cdot (\hat{u}_h-u_h)^{\tau}\,dl\\
&+ \frac{\overline{\nu}\alpha k^2}{h}\int_{\partial T} (\hat{u}_h-u_h)^{\tau}\cdot (\hat{v}_h-v_h)^{\tau}\,dl,
\end{align*}
where $\mu$ is the co-normal vector, $\alpha>0$ a stabilization parameter, $h$ the mesh-size, $k$ the used polynomial degree, and $(u)^{\tau}:= u-(u\cdot\mu)\mu$ the projection onto the tangent vector of the element boundary edge.

For the pressure discontinous elements $Q_h^{k-1}\subset L^2(S)$ of one polynomial order less than the velocity are used to guarantee exactly divergence-free velocity fields. The divergence-constraint thus reads

\begin{align*}
b_h(\tilde{u}_h,q_h)=-\int_S \mathrm{div}_S(u_h)\,q_h\,ds.
\end{align*}

For the convection term upwinding is used

\begin{align*}
    c_h(\tilde{w}_h,\tilde{u}_h,\tilde{v}_h)=\sum_{T\in \mathcal{T}_h} -\int_T u_h\otimes w_h: \nabla_S v_h\,ds + \int_{\partial T} w_h\cdot\mu\, u_h^{\mathrm{up}}\cdot v_h\,dl+\int_{\partial T_{\mathrm{out}}}w_h\cdot\mu \,(\hat{u}_h-u_h)^{\tau}\cdot \hat{v}_h\,dl,
\end{align*}
where 

\begin{align*}
u_h^{\mathrm{up}} = (u_h\cdot\mu)\mu + \begin{cases} (u_h\cdot \tau)\tau & \text{ if }w_h\cdot\mu >0,\\ (\hat{u}_h\cdot\tau)\tau & \text{ else }.\end{cases}
\end{align*}

<img src="pictures/upwind_triangles.jpg" width="150" align="center"> 

The mass bilinear form is simply

\begin{align*}
m_h(\tilde{u}_h,\tilde{v}_h)=\int_S u_h\cdot v_h\,ds.
\end{align*}

An IMEX is used for time discretization, where stiffness and divergence constraint is handles implicitly, convection explicitly.


\begin{align*}
&m_h(\frac{\tilde{u}_h^{n+1}-\tilde{u}_h^n}{\Delta t},\tilde{v}_h) + a_h(\tilde{u}^{n+1}_h,\tilde{v}_h) + c_h(\tilde{u}_h^{n},\tilde{u}_h^{n},\tilde{v}_h) + b_h(\tilde{v}_h,\tilde{p}_h^{n+1}) = f^n(\tilde{v}_h),\\
&b_h(\tilde{u}_h^{n+1},\tilde{q}_h) = 0.
\end{align*}

In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
import ipywidgets as widgets

Create geometry and mesh

In [ ]:
r = 2 / pi

arc = ArcOfCircle((0, 0.41, 0), (r, 0.41, r), (2 * r, 0.41, 0))
shell = Prism(arc, (0, -0.41, 0.0))

holecoorx = cos(0.2 / r) * r
holecoorz = sin(0.2 / r) * r
hole = Cylinder((r, 0.2, 0), -holecoorx * X + holecoorz * Z, r=0.05, h=2)
surf = shell - hole

surf.edges.name = "wall"
surf.edges.Min(X).name = "inlet"
surf.edges.Max(X).name = "outlet"

mesh = Mesh(OCCGeometry(surf).GenerateMesh(maxh=0.1)).Curve(2)
Draw(mesh);

Curve mesh, define FESpaces and test- and trialfunctions

In [ ]:
order = 3
mesh.Curve(order)

# penalty parameter
alpha = 5
# viscosity
nu = 0.001
# time-step
tau = 0.0005

VDivSurf = HDivSurface(mesh, order=order, dirichlet_bbnd="inlet|wall")
VHat = HCurl(mesh, order=order, orderface=0, dirichlet_bbnd="inlet|wall")
Q = SurfaceL2(mesh, order=order - 1)
V = VDivSurf * VHat * Q

(u, uhat, p), (v, vhat, q) = V.TnT()
uhat, vhat, p, q = uhat.Trace(), vhat.Trace(), p.Trace(), q.Trace()

Define outer normal vector $\nu$, edge tangential vector $\tau$, element-normal (co-normal) vector $\mu=\nu\times \tau$, projection to edge tangent and surface tangent space $\boldsymbol{P}$, and mesh-size $h$.

In [ ]:
normal = specialcf.normal(3)
tangential = specialcf.tangential(3)
n = Cross(normal, tangential)


def tang(vec):
    return vec - (vec * n) * n


P = Id(3) - OuterProduct(normal, normal)

h = specialcf.mesh_size

Define mass, stiffness + divergence constraint, and convection bilinear forms.

In [ ]:
m = BilinearForm(u.Trace() * v.Trace() * ds, symmetric=True).Assemble()


a = BilinearForm(V, symmetric=True)
a += nu * InnerProduct(Sym(P * Grad(u)), Sym(P * Grad(v))) * ds
a += (
    nu
    * InnerProduct(Sym(P * Grad(u)) * n, tang(vhat - v.Trace()))
    * ds(element_boundary=True)
)
a += (
    nu
    * InnerProduct(Sym(P * Grad(v)) * n, tang(uhat - u.Trace()))
    * ds(element_boundary=True)
)
a += (
    nu
    * alpha
    * order
    * order
    / h
    * InnerProduct(tang(vhat - v.Trace()), tang(uhat - u.Trace()))
    * ds(element_boundary=True)
)
a += (-div(u.Trace()) * q - div(v.Trace()) * p) * ds
a.Assemble()


conv = BilinearForm(V, nonassemble=True)
conv += -InnerProduct(P * Grad(v) * u.Trace(), u.Trace()) * ds
u_Other = (u.Trace() * n) * n + tang(uhat)
conv += IfPos(
    u.Trace() * n,
    u.Trace() * n * u.Trace() * v.Trace(),
    u.Trace() * n * u_Other * v.Trace(),
) * ds(element_boundary=True)

# zero right-hand side
f = LinearForm(V).Assemble()

convvec = f.vec.CreateVector()

Solve surface Stokes problem to extend inflow velocity as initial data for Navier-Stokes.

In [ ]:
U0 = 1.5
uin = CF(U0 * 4 * y * (0.41 - y) / (0.41 * 0.41) * IfPos(1e-2 - x, 1, 0))
dirvals = CF((0, 0, uin))

gfu = GridFunction(V)
gfu.components[0].Set(dirvals, definedon=mesh.Boundaries(".*"))
gfu.components[1].Set(dirvals, definedon=mesh.Boundaries(".*"))

invstokes = a.mat.Inverse(V.FreeDofs(), inverse="")
res = gfu.vec.CreateVector()
res.data = f.vec - a.mat * gfu.vec
gfu.vec.data += invstokes * res

Draw(gfu.components[0], mesh, "velocity")
Draw(div(gfu.components[0]), mesh, "divergence");

Prepare the $M$-star matrix for the IMEX time-stepping scheme.

In [ ]:
mstar = m.mat.CreateMatrix()
mstar.AsVector().data = m.mat.AsVector() + tau * a.mat.AsVector()
inv = mstar.Inverse(V.FreeDofs(), inverse="")

scene_u = Draw(gfu.components[0], mesh, "velocity", order=3)
scene_p = Draw(gfu.components[2], mesh, "pressure", order=3);

In [ ]:
tend = 6
t = 0
i = 0

tw = widgets.Text(value="t = 0")
display(tw)

with TaskManager():
    while t < tend:
        conv.Apply(gfu.vec, convvec)
        res.data = a.mat * gfu.vec + convvec
        gfu.vec.data -= tau * inv * res

        if i % 20 == 0:
            scene_u.Redraw()
            scene_p.Redraw()

        i += 1
        t += tau
        tw.value = f"t = {round(t,6)}"